In [ ]:
# import the required libraries
from czitools import pylibczirw_metadata as czimd
from czitools import pylibczirw_tools as czird
from czitools import misc, napari_tools
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
import napari
from pathlib import Path
import os

In [ ]:
defaultdir = os.path.join(Path(os.getcwd()).resolve().parents[1], "data")
fc = FileChooser(defaultdir)
fc.filter_pattern = '*.czi'
display(fc)

In [ ]:
# Print the selected path, filename, or both
print(fc.selected_path)
print(fc.selected_filename)
print(fc.selected)

In [ ]:
# get only specific metadata
czi_dimensions = czimd.CziDimensions(fc.selected)
print("SizeS: ", czi_dimensions.SizeS)
print("SizeT: ", czi_dimensions.SizeT)
print("SizeZ: ", czi_dimensions.SizeZ)
print("SizeC: ", czi_dimensions.SizeC)
print("SizeY: ", czi_dimensions.SizeY)
print("SizeX: ", czi_dimensions.SizeX)

In [ ]:
# and get more info
czi_scaling = czimd.CziScaling(fc.selected)
czi_channels = czimd.CziChannelInfo(fc.selected)
czi_bbox = czimd.CziBoundingBox(fc.selected)
czi_objectives = czimd.CziObjectives(fc.selected)
czi_detectors = czimd.CziDetector(fc.selected)
czi_microscope = czimd.CziMicroscope(fc.selected)
czi_sample = czimd.CziSampleInfo(fc.selected)

In [ ]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(fc.selected)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_mdict_red(mdata, sort=False)

# convert metadata dictionary to a pandas dataframe
mdframe = misc.md2dataframe(mdict)

# and display it nicely as a HTML inside the jupyter notebook
display(HTML(mdframe.to_html()))

In [ ]:
# write XML to disk
xmlfile = czimd.writexml(fc.selected)
print("XML File written to:", xmlfile)

In [ ]:
# get the planetable for the CZI file
pt, csvfile = misc.get_planetable(fc.selected,
                                  norm_time=True,
                                  savetable=True,
                                  separator=',',
                                  index=True)

# and display it nicely as a HTML inside the jupyter notebook
display(HTML(pt.to_html()))
print("PlaneTable CSV File written to:", csvfile)

In [ ]:
# return array with dimension order STZCYXA
array6d, mdata, dimstring = czird.read_6darray(fc.selected,
                                        dimorder="STCZYX",
                                        output_dask=False,
                                        remove_Adim=True)

# remove A dimension do display the array inside Napari
dim_order, dim_index, dim_valid = czimd.CziMetadata.get_dimorder(dimstring)

In [ ]:
# show array inside napari viewer
viewer = napari.Viewer()
layers = napari_tools.show(viewer, array6d, mdata,
                           dim_string=dimstring,
                           blending="additive",
                           contrast='napari_auto',
                           gamma=0.85,
                           add_mdtable=True,
                           name_sliders=True)

In [ ]:
napari.utils.nbscreenshot(viewer)